In [18]:
%load_ext autoreload
%autoreload 2
import os

from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import Cityscapes
from torchvision.utils import save_image

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
# make a folder to save output images
from unet import UNet
if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')

# a function used to transform numpy array to image format
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), x.size(1), x.size(2), x.size(3))
    return x

#################### select your hyperparameters ############################
num_epochs = 10
batch_size = 1
n_samples = 20

####### define image transforms, you can have other choices, explore it! #####
transform = transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(),
])

# download dataset
dataset = Cityscapes('./data/cityscapes', transform=transform, target_transform=transform, target_type='color')
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [20]:
model = UNet(n_channels=3, n_classes=1)
############ chose appropriate loss function ######################
criterion = nn.L1Loss()

# set the optimizer, explore the effect of different optimizers
lr = 0.1
optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=1e-8)

In [21]:
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader):
        if i > n_samples:
            break
        img, mask_with_alpha = data
        mask = mask_with_alpha[:, :3, :, :]
        #img = img.view(img.size(0), -1)
        # ===================forward=====================
        img_pred = model(mask)
        loss = criterion(img_pred, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================user interaction========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data))
    if epoch % 1 == 0:
        pic = to_img(img_pred.data)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))
        
        ori_pic = to_img(img.data)
        save_image(ori_pic, './mlp_img/ori_image_{}.png'.format(epoch))
        
        mask_pic = to_img(mask.data)
        save_image(mask_pic, './mlp_img/mask_image_{}.png'.format(epoch))


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:88: UserWarning: Using a target size (torch.Size([1, 3, 128, 256])) that is different to the input size (torch.Size([1, 1, 128, 256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


epoch [1/10], loss:0.2511
epoch [2/10], loss:0.1688
epoch [3/10], loss:0.1305
epoch [4/10], loss:0.1263
epoch [5/10], loss:0.0783
epoch [6/10], loss:0.1148
epoch [7/10], loss:0.1552
epoch [8/10], loss:0.1103
epoch [9/10], loss:0.1121
epoch [10/10], loss:0.0772
